In [1]:
from imports_func import *
from dbase import *

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
X=np.load('/home/miquel/ForMiquel/CLEAN2/X_full.npy') 
y=np.load('/home/miquel/ForMiquel/CLEAN2/y_1j_full.npy')
jac_idx=np.load('/home/miquel/ForMiquel/CLEAN2/jac_idx_1j_full.npy')
sub_id=np.load('/home/miquel/ForMiquel/CLEAN2/sub_id_short.npy') #subject idxs

In [3]:
# build vector with dt's
dt=[]
for i in range(0,len(sub_id)):
    dt.append(Subjects[sub_id[i]].jacobians[jac_idx[i]].dt)
dt = np.asarray(dt)/365

In [4]:
# build vector with ages
ages=[]
for i in range(0,len(sub_id)):
    ages.append(Subjects[sub_id[i]].age_bl)
    
ages = np.asarray(ages) 

In [45]:
def cross_validation_analysis(X, y, zeros, clf, splits, percentage, dt):
      
    split=0
    box = np.zeros((121,145,121))
    precision_l=[]
    recall_l=[]
    threshold_l=[]
    print("split:",split)
    

    spl = StratifiedShuffleSplit(n_splits=splits, test_size=0.2, random_state=40)
    for train_index, test_index in spl.split(X, y):
        X_train = X[train_index,:]
        y_train = y[train_index]
        X_test = X[test_index,:]
        y_test = y[test_index]
        
        spl_aux = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=40)
        for train_index, test_index in spl_aux.split(X_train, y_train):
            X_train_aux = X_train[train_index,:]
            y_train_aux = y_train[train_index]
            X_test_aux = X_train[test_index,:]
            y_test_aux = y_train[test_index]

            S = SelectPercentile(percentile=percentage)
            S.fit(X_train_aux, y_train_aux)
        
            X_train_aux_s = X_train_aux[:,S.get_support()]
            X_test_aux_s = X_test_aux[:,S.get_support()]
            X_test_s = X_test[:,S.get_support()]
        
            if split == 0:
                print("shape X_train:",np.shape(X_train_aux_s))
        
            model = clf
            y_score_train = model.fit(X_train_aux_s, y_train_aux).decision_function(X_train_aux_s)
            model = clf
            y_score = model.fit(X_train_aux_s, y_train_aux).decision_function(X_test_aux_s)
        
      
        #mask0 = np.where(dt_train>400)
        #mask0 = np.asarray(mask0)
        #y_train = y_train[mask0][0]
        #y_score_train = y_score_train[mask0][0]
    
        #mask0 = np.where(dt_test>400)
        #mask0 = np.asarray(mask0)
        #y_test = y_test[mask0][0]
        #y_score = y_score[mask0][0]
        
            average_precision_train = average_precision_score(y_train_aux, y_score_train)
            average_precision = average_precision_score(y_test_aux, y_score)
            print("average_precision_train:", average_precision_train)
            print("average_precision_test:", average_precision)
        
            precision_t, recall_t, thresholds_t = precision_recall_curve(y_train_aux, y_score_train) 
            precision, recall, thresholds = precision_recall_curve(y_test_aux, y_score)    
        
            f1 = 2 * (precision * recall) / (precision + recall)
    
            maxi = np.amax(f1)
            maxi_idx = np.argmax(f1)
        
            if precision[maxi_idx] != 0 or recall[maxi_idx] != 0:
                
                th_opt=thresholds[maxi_idx]#chosen th with train
                print("th_opt:", th_opt)
                y_score = model.fit(X_train_aux_s, y_train_aux).decision_function(X_test_s)
                print("y_score:", y_score)
                mask = np.where(y_score>th_opt)#1's in y_true test
                mask = np.asarray(mask)
                pred=np.zeros(X_test.shape[0])
                pred[mask[0]]=np.ones(len(mask[0]))
                print("pred:", pred)
                precision=precision_score(y_test, pred)
                recall=recall_score(y_test, pred)
                
                
                precision_l.append(precision)
                recall_l.append(recall)
                threshold_l.append(th_opt)
                print("split:",split)
                split = split + 1
                

    
    print("thresholds:", threshold_l)
    print()
    print("precisions:", precision_l)
    print()
    print("precision mean:", np.mean(precision_l))
    print()
    print("recalls:", recall_l)
    print()
    print("recall mean:", np.mean(recall_l))        

    
    

In [46]:
zeros = np.load('/home/miquel/ForMiquel/CLEAN2/zero_cols_JB_full.npy') 
#X=np.transpose(np.transpose(X)/dt)
clf = LogisticRegression(solver='liblinear', penalty='l2', class_weight='balanced', C=1.25)
splits=40
percentage=0.25
age_dt=0 #1 if we want to add normalized age and dt as features
cross_validation_analysis(X, y, zeros, clf, splits, percentage, dt)

split: 0


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    13     99    177 ..., 573915 573916 573919] are constant.
  UserWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


shape X_train: (79, 1435)
average_precision_train: 0.860914117583
average_precision_test: 0.720455473856
th_opt: -0.52601316158
y_score: [ 0.04551051 -0.17826228 -0.49137023  1.55849549 -0.212661   -0.51677817
 -0.12080114  2.09592309  0.08924537  0.20989127 -0.4174844  -0.07809308
 -0.50153372 -0.43234391 -0.64902184  0.44388553 -0.280707   -0.53210853
 -0.6636579   0.26163695 -0.49280024 -1.34164748 -1.23814861 -0.84535561
 -0.62116336]
pred: [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  0.
  0.  1.  1.  0.  0.  0.  0.]
split: 0


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [   178    179    183 ..., 573917 573920 573921] are constant.
  UserWarning)


average_precision_train: 0.940458824293
average_precision_test: 0.708807174185
th_opt: -0.0941158689059
y_score: [ -4.49614290e-01  -3.70506411e-01  -3.94185055e-01  -3.43639795e-01
  -1.53697305e-01  -3.41107381e-01  -5.25537162e-01   5.48280866e-01
  -5.10501494e-01  -3.09119407e-01  -4.76697499e-01  -1.41712463e-01
  -2.30472501e-01   4.28826302e-01  -2.52962291e-01  -2.56601949e-01
  -1.95579008e-04  -2.58493474e-01   1.60141621e+00  -1.88726971e-01
  -8.80580226e-02   2.15921901e-01  -5.48019183e-01  -2.78104435e-01
   6.81454557e-01]
pred: [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.
  1.  0.  1.  1.  0.  0.  1.]
split: 1


...

average_precision_train: 0.904858538198
average_precision_test: 0.857633053221
th_opt: 0.29857192102
y_score: [ 0.94217727 -0.29185049  0.3137392  -0.08800771 -0.32260764 -0.06491671
 -0.36082506 -0.1046879   0.2116921  -0.45611497 -0.36088087 -0.23210332
 -0.45865397  0.22524933  0.67339172 -0.41628838  0.56479035 -0.45471631
 -0.24873059  0.40226326 -0.66945905  0.29038894 -0.60992496 -0.64497434
 -0.30644165]
pred: [ 1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.
  0.  1.  0.  0.  0.  0.  0.]
split: 29


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [   196    265    275 ..., 573877 573887 573888] are constant.
  UserWarning)


average_precision_train: 0.934722547884
average_precision_test: 0.804247835498
th_opt: -0.172246690791
y_score: [-0.1261013  -0.17673517  0.79159962  0.27084151  0.20074134 -1.42652807
  0.1052503  -0.08114772 -0.10136448 -0.30669287  0.36068538 -2.46250833
 -0.31943969  0.1714563   0.94668881 -0.2833544  -1.09130101  0.09108489
  0.49319825  0.13436941  0.11215833  0.31964002 -0.0053948  -0.02410406
  1.29770333]
pred: [ 1.  0.  1.  1.  1.  0.  1.  1.  1.  0.  1.  0.  0.  1.  1.  0.  0.  1.
  1.  1.  1.  1.  1.  1.  1.]
split: 30


/usr/local/lib/python3.4/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [    13     99    149 ..., 573912 573913 573914] are constant.
  UserWarning)


average_precision_train: 0.948311554289
average_precision_test: 0.759972848789
th_opt: -0.014856234022
y_score: [ 0.66018221 -0.01831602  0.56571378 -0.64732653 -0.33007255 -0.22598909
 -0.28277748 -0.3486343  -0.1670047  -0.25085824 -0.17221325 -0.25157017
 -0.06805102 -0.25964039 -0.31699925 -0.17429211 -0.22334014 -0.20166293
  0.05953312 -0.40559642 -0.30402997 -0.2288185   0.34529846 -0.10607688
  0.072181  ]
pred: [ 1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  1.  0.  1.]
split: 31
thresholds: [-0.52601316158033695, -0.094115868905930933, -1.160074536492079, -0.75038482515443339, -0.21228560760927939, -0.13693608931888118, -1.1800478745517702, -1.1600943115634719, -0.20836418927549361, -1.4950324384085156, -1.2882028664705532, 0.066955431898712781, -1.4459471919631435, -0.76185334638352964, -2.3315002262777274, 0.019048806889031947, -1.7841100037792235, -0.90861281393670179, 0.016767066374483219, -0.09110403169234671, -0.034429197264441

In [ ]:
# Split the dataset
spl = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=39)
for train_index, test_index in spl.split(X, y): #split in subjects
    X_train = X[train_index,:]
    X_test = X[test_index,:]
    y_train = y[train_index]
    y_test = y[test_index]

C = []
#C = [0.01,10]
acc_linear = []
roc_linear = []
f1_linear = []
mcc_linear = []


for i in C:
    #clf = SVC(kernel='linear', probability = True, C=i)
    clf = LogisticRegression(solver='liblinear', penalty='l2', C=i, class_weight='balanced')

    y_score = clf.fit(X_train, y_train).decision_function(X_test)
    pred = clf.predict(X_test)
    acc = accuracy_score(y_test, pred)
    roc=roc_auc_score(y_test, y_score)
    f1=f1_score(y_test, pred)
    from sklearn.metrics import matthews_corrcoef
    mcc=matthews_corrcoef(y_test, pred)
    print(confusion_matrix(y_test, pred))

    acc_linear.append(acc)
    roc_linear.append(roc)
    f1_linear.append(f1)
    mcc_linear.append(mcc)

plt.figure(1)
plt.plot(C, acc_linear, color='green', label='acc')
plt.plot(C, roc_linear, color='red', label='roc')
plt.plot(C, f1_linear, color='blue', label='f1_score')
plt.plot(C, mcc_linear, color='orange', label='mcc')

plt.legend(loc='best')
plt.xlabel('C')
plt.title('metrics logistic regression')
plt.show()


In [47]:
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)


# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Set the parameters by cross-validation
tuned_parameters = [{'C': [0.05,0.25,0.5,0.75,1,1.25,1.5,1.75,2]}, ]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(LogisticRegression(solver='liblinear', penalty='l2',class_weight='balanced'), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 0.05}

Grid scores on development set:

0.560 (+/-0.207) for {'C': 0.05}
0.512 (+/-0.266) for {'C': 0.25}
0.528 (+/-0.282) for {'C': 0.5}
0.515 (+/-0.272) for {'C': 0.75}
0.515 (+/-0.272) for {'C': 1}
0.535 (+/-0.285) for {'C': 1.25}
0.546 (+/-0.269) for {'C': 1.5}
0.546 (+/-0.269) for {'C': 1.75}
0.546 (+/-0.269) for {'C': 2}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       0.82      0.52      0.64        27
          1       0.38      0.73      0.50        11

avg / total       0.70      0.58      0.60        38


# Tuning hyper-parameters for recall

Best parameters set found on development set:

{'C': 1.5}

Grid scores on development set:

0.540 (+/-0.131) for {'C': 0.